In [1]:
import pandas as pd
import numpy as np

In [66]:
topics = pd.read_excel("tweet_topics.xlsx")
candidates = pd.read_excel("../Senator_candidates.xlsx")
tweets = pd.read_excel("../merged/tweets.xlsx")
users = pd.read_excel("../merged/users.xlsx")

C:\Users\Arushi Jain\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [67]:
def combine_topics(topics):
    mergeTopics = pd.DataFrame()
    mergeTopics.index = topics.index
    mergeTopics["Election"] = topics[0] + topics[9] + topics[10] + topics[12] + topics[13] + topics[14]+ topics[17]
    mergeTopics["Crime/ Race"] = topics[1] + topics[8]
    mergeTopics["Inflation"] = topics[2] + topics[4] + topics[6]
    mergeTopics["Healthcare"] = topics[3]
    mergeTopics["Abortion/ Women Rights"] = topics[5] + topics[16]
    mergeTopics["Student/ Schools"] = topics[19]
    mergeTopics["Support"] = topics[7] + topics[11] + topics[15] + topics[18]
    return mergeTopics

In [68]:
original_topics = topics.copy(deep=True)
topics = topics.fillna(0)
topics = topics.mask(topics < 0.05, 0)
topics = topics.set_index("tweetId")
topics = combine_topics(topics)
topics

,Election,Crime/ Race,Inflation,Healthcare,Abortion/ Women Rights,Student/ Schools,Support
tweetId,,,,,,,
1582199030079102976,0.352472,0.000000,0.000000,0.000000,0.391523,0.0,0.000000
1582174634480280064,0.805296,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1581788310740951040,0.758889,0.000000,0.000000,0.000000,0.000000,0.0,0.078348
1581759471558095104,0.150237,0.240212,0.260763,0.067429,0.071013,0.0,0.062772
1581733146562768896,0.486184,0.000000,0.000000,0.077545,0.269208,0.0,0.000000
...,...,...,...,...,...,...,...
1579289579378523904,0.501665,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1579288347012665088,0.572800,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1578249864152219904,0.590915,0.000000,0.081690,0.000000,0.000000,0.0,0.151184


In [69]:
candidates = candidates.drop(labels='Column1', axis=1)
cList = users["username"].tolist()
candidates = candidates[candidates["TwitterId"].isin(cList)]

In [70]:
candidates = candidates.set_index("TwitterId")
users = users.set_index("username")
tweets = tweets.set_index("tweetId")

In [71]:
candidates["followers"] = users["followers_count"]

In [72]:
candidates["avgLikes"] = tweets[["username", "favorites"]].groupby("username").mean()
candidates["avgRetweets"] = tweets[["username", "retweets"]].groupby("username").mean()

In [73]:
candidates

,State,Candidate,Party,Currently Serving,followers,avgLikes,avgRetweets
TwitterId,,,,,,,
AdamLaxalt,Nevada,Adam Laxalt,Republican,0,60766,236.163265,79.112245
bgmasters,Arizona,Blake Masters,Republican,0,175270,1099.652695,273.197605
Booker4KY,Kentucky,Charles Booker,Democrat,0,452394,5835.035714,1289.321429
BrianBengs,South Dakota,Brian Bengs,Democrat,0,44577,516.925926,206.120370
brianschatz,Hawaii,Brian Schatz,Democrat,1,412852,778.636905,129.857143
...,...,...,...,...,...,...,...
SenToddYoung,Indiana,Todd Young,Republican,1,70842,27.485294,6.132353
Stephen_Faris,Ohio,Stephen Faris,Independent,0,191,0.750000,0.250000
tommcdermottjr,Indiana,Thomas McDermott Jr.,Democrat,0,10524,52.676056,13.816901


In [83]:
def compute_weighted_sentiment(orig_topics, tweets, candidates):
    topics = orig_topics.copy(deep=True)
    topics['username'] = tweets['username']
    topics['avgSentiment'] = tweets['avgSentiment']
    weight_list = []
    for candidate in candidates:        
        c_dict = {}
        c_dict['username'] = candidate
        topics_df = topics[topics['username']==candidate]
        for topic in topics.columns[:7]:
            weight = (topics_df[topic]*topics_df["avgSentiment"]).mean()
            c_dict[topic] = weight
        weight_list.append(c_dict)
    return pd.DataFrame(weight_list).set_index("username")
            
        

In [75]:
def process_reply_sentiments(sentiment, score_field, model):
    # Assign senators to each reply
    sen = []
    for idx, row in sentiments.iterrows():
        sen.append(tweets.at[row['replyTo'], 'username'])
    sentiments['username'] = sen

    # Compute average sentiment per senator from reply sentiments
    expectedSentiment = []
    avgSentimentSenator = sentiments[[score_field, 'username']].groupby('username').mean()
    
    # Compute Expected Sentiment on each tweet
    for idx, row in tweets.iterrows():
        if row['username'] not in avgSentimentSenator.index:
            expectedSentiment.append(0)
            continue
        expectedSentiment.append(avgSentimentSenator.at[row['username'], score_field] * row['replies'])
    tweets['avgSentiment'] = expectedSentiment
    
    # Assign Sentiment Weights to Topics
    senator_topics = compute_weighted_sentiment(topics, tweets, cList)
    print(senator_topics.describe())
    
    # SAVE
    candidates_df = candidates.merge(senator_topics, left_index=True, right_index=True, how='inner')
    candidates_df.to_excel(f"Features_{model}.xlsx")

In [84]:
sentiments = pd.read_excel("../Anant/Sentiment Analysis/Replies Dataset.xlsx")
sentiments = sentiments[['tweetId', 'replyTo', 'Effective Score', 'Effective Sentiment']]
#sentiments['Sentiment Score'] = sentiments.apply(lambda x: -x['Sentiment Score'] if x['Sentiment']=='NEGATIVE' else x['Sentiment Score'], axis=1)
#sentiments = sentiments.drop(labels="Sentiment", axis=1)
process_reply_sentiments(sentiments, "Effective Score", "Bert")

         Election  Crime/ Race   Inflation  Healthcare  \
count   88.000000    88.000000   88.000000   88.000000   
mean   -13.995283    -3.217003   -8.274953   -1.557306   
std     39.567333     7.889653   35.437451    3.852388   
min   -280.731550   -52.712069 -322.777590  -24.882851   
25%     -8.012861    -2.490622   -4.801624   -1.083864   
50%     -2.281437    -0.482227   -0.760489   -0.246872   
75%     -0.162473    -0.039685   -0.084165   -0.015376   
max      0.135350     0.012357    0.047563    0.001820   

       Abortion/ Women Rights  Student/ Schools     Support  
count               88.000000         88.000000   88.000000  
mean                -3.511259         -0.983031   -7.526082  
std                 11.185530          3.375918   27.545305  
min                -96.176518        -28.905588 -248.034726  
25%                 -1.715862         -0.431601   -4.573875  
50%                 -0.522805         -0.172500   -1.169802  
75%                 -0.035806         -0.00

In [85]:
sentiments = pd.read_excel("../Nikos/blob.xlsx")
sentiments = sentiments[['tweetId', 'replyTo', 'blob_score', 'sentiment_blob']]
process_reply_sentiments(sentiments, "blob_score", "Blob")

        Election  Crime/ Race  Inflation  Healthcare  Abortion/ Women Rights  \
count  88.000000    88.000000  88.000000   88.000000               88.000000   
mean    0.545898     0.168786   0.314768    0.070743                0.172737   
std     1.367167     0.321662   0.927251    0.174091                0.372413   
min    -5.405367    -0.155044  -1.526284   -0.770828               -0.420580   
25%     0.033025     0.008376   0.013187    0.001705                0.008051   
50%     0.171143     0.040233   0.080548    0.022008                0.037619   
75%     0.491353     0.112012   0.252150    0.062146                0.123637   
max     6.689838     1.825910   7.691796    0.901332                2.291888   

       Student/ Schools    Support  
count         88.000000  88.000000  
mean           0.049575   0.331223  
std            0.122965   0.800672  
min            0.000000  -1.404755  
25%            0.002094   0.016642  
50%            0.012133   0.091157  
75%            0.041

In [86]:
sentiments = pd.read_excel("../Nikos/flair.xlsx")
sentiments = sentiments[['tweetId', 'replyTo', 'flair_score', 'sentiment_flair']]
process_reply_sentiments(sentiments, "flair_score", "Flair")

         Election  Crime/ Race   Inflation  Healthcare  \
count   88.000000    88.000000   88.000000   88.000000   
mean    -9.185670    -2.111094   -5.475334   -0.993999   
std     27.262368     5.715142   24.239510    2.594601   
min   -191.526786   -40.269382 -220.212350  -17.807321   
25%     -3.870381    -1.498302   -2.516907   -0.674497   
50%     -1.093940    -0.228167   -0.410958   -0.144519   
75%     -0.095128    -0.027139   -0.043889   -0.009421   
max      0.190850     0.017424    0.067066    0.003255   

       Abortion/ Women Rights  Student/ Schools     Support  
count               88.000000         88.000000   88.000000  
mean                -2.284983         -0.645453   -4.910757  
std                  7.636391          2.317497   18.818973  
min                -65.615636        -19.720599 -169.219647  
25%                 -0.826940         -0.217716   -2.331755  
50%                 -0.282020         -0.089923   -0.562456  
75%                 -0.023377         -0.00